In [1]:
import sys
sys.path.insert(0, "ladybug")
sys.path.insert(0, "honeybee")

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [3]:
import ladybug
import honeybee

import json
import itertools
import re
import os

import eppy
from eppy import modeleditor
from eppy.modeleditor import IDF
import numpy

In [13]:
# Define a function to load a json file into a dictionary object
def loadJSON(path):
    import json
    with open(path) as data_file:
        return json.load(data_file)

In [10]:
# Load IDF ready for pre-processing and modification
idf_file = "test.idf"
epw_file = "weather/GBR_Cardiff_CIBSE_TRY.epw" 
idd_file = "C:/EnergyPlusV8-8-0/Energy+.idd" # "/Applications/EnergyPlus-8-8-0/Energy+.idd" "C:/EnergyPlusV8-8-0/Energy+.idd"

IDF.setiddname(idd_file)
idf = IDF(idf_file)

In [11]:
# Parameters actually containing values 
[i for i in idf.idfobjects if len(idf.idfobjects[i]) != 0]

[u'SHADING:BUILDING:DETAILED',
 u'VERSION',
 u'OUTPUTCONTROL:TABLE:STYLE',
 u'SIZING:PARAMETERS',
 u'LIGHTS',
 u'WINDOWMATERIAL:GAS',
 u'MATERIAL',
 u'SIMULATIONCONTROL',
 u'MATERIAL:AIRGAP',
 u'DESIGNSPECIFICATION:OUTDOORAIR',
 u'PEOPLE',
 u'RUNPERIOD',
 u'SCHEDULE:WEEK:DAILY',
 u'ZONEINFILTRATION:DESIGNFLOWRATE',
 u'ZONE',
 u'ELECTRICEQUIPMENT',
 u'SIZINGPERIOD:DESIGNDAY',
 u'ZONEMIXING',
 u'OUTPUT:VARIABLEDICTIONARY',
 u'HVACTEMPLATE:THERMOSTAT',
 u'BUILDINGSURFACE:DETAILED',
 u'SITE:LOCATION',
 u'SCHEDULE:YEAR',
 u'FENESTRATIONSURFACE:DETAILED',
 u'TIMESTEP',
 u'GLOBALGEOMETRYRULES',
 u'SCHEDULE:DAY:INTERVAL',
 u'OUTPUT:SURFACES:LIST',
 u'HVACTEMPLATE:ZONE:IDEALLOADSAIRSYSTEM',
 u'CONSTRUCTION',
 u'OUTPUT:VARIABLE',
 u'SHADOWCALCULATION',
 u'WINDOWMATERIAL:GLAZING',
 u'BUILDING',
 u'SCHEDULETYPELIMITS']

In [12]:
#Load the EPW file to get the location variables and store in the IDF object
with open(epw_file, "r") as f:
    a, b, c, d, e, f, g, h, i, j = f.readlines()[0].replace("\n", "").split(",")
idf.idfobjects["SITE:LOCATION"] = []
idf.newidfobject('SITE:LOCATION',  Name=b, Latitude=float(g), Longitude=float(h), Time_Zone=float(i), Elevation=float(j),)

# Set version number
idf.idfobjects["VERSION"] = []
idf.newidfobject('VERSION', Version_Identifier="8-8-0")

# Set output variables for each zone simulated
output_variables = [
    "Zone Mean Air Temperature",
    "Zone Mean Radiant Temperature",
    "Zone Air Relative Humidity",
    "Zone Operative Temperature",
    "Zone People Total Heating Energy",
    "Zone Lights Electric Energy",
    "Zone Electric Equipment Electric Energy",
    "Zone Windows Total Transmitted Solar Radiation Energy",
    "Zone Ideal Loads Zone Total Heating Energy",
    "Zone Ideal Loads Zone Total Cooling Energy",
    "Zone Ideal Loads Supply Air Total Cooling Energy",
    "Zone Ideal Loads Supply Air Total Heating Energy",
    "Zone Ventilation Sensible Heat Loss Energy",
    "Zone Ventilation Sensible Heat Gain Energy",
    "Zone Ventilation Fan Electric Energy",
]
idf.idfobjects["OUTPUT:VARIABLE"] = []
[idf.newidfobject('OUTPUT:VARIABLE', Key_Value="*", Variable_Name=i, Reporting_Frequency="hourly") for i in output_variables]

# Remove Design Day sizing periods
idf.idfobjects["SIZINGPERIOD:DESIGNDAY"] = []

# Remove surface output (to save on simulation time and results size)
idf.idfobjects['OUTPUT:SURFACES:LIST'] = []

# Remove table style output to save on results file size
idf.idfobjects["OUTPUTCONTROL:TABLE:STYLE"] = []

# Remove sizing parameters
idf.idfobjects["SIZING:PARAMETERS"] = []

# Set simulation to run only for annual period corresponding with weatherfile
idf.idfobjects["SIMULATIONCONTROL"] = []
idf.newidfobject("SIMULATIONCONTROL", Do_Zone_Sizing_Calculation="No", Do_System_Sizing_Calculation="No", Do_Plant_Sizing_Calculation="No", Run_Simulation_for_Sizing_Periods="No", Run_Simulation_for_Weather_File_Run_Periods="Yes")

# Set simulation run period (including start day of year)
idf.idfobjects["RUNPERIOD"] = []
idf.newidfobject("RUNPERIOD", Name="Custom Run", Begin_Month=1, Begin_Day_of_Month=1, End_Month=12, End_Day_of_Month=31, Day_of_Week_for_Start_Day="Monday", Use_Weather_File_Holidays_and_Special_Days="Yes", Use_Weather_File_Daylight_Saving_Period="Yes", Apply_Weekend_Holiday_Rule="No", Use_Weather_File_Rain_Indicators="Yes", Use_Weather_File_Snow_Indicators="Yes")

# Remove output variable dictionary
idf.idfobjects["OUTPUT:VARIABLEDICTIONARY"] = []

# Set general building parameters (including North angle)
idf.idfobjects["BUILDING"] = []
idf.newidfobject("BUILDING", Name="IDF Name", North_Axis=0, Terrain="City", Solar_Distribution="FullExteriorWithReflections", Maximum_Number_of_Warmup_Days=25, Minimum_Number_of_Warmup_Days=6)

# Set number of timesteps per hour in simulation
idf.idfobjects["TIMESTEP"] = []
idf.newidfobject("TIMESTEP", Number_of_Timesteps_per_Hour=6)

# Set shadow calculation method
idf.idfobjects["SHADOWCALCULATION"] = []
idf.newidfobject("SHADOWCALCULATION", Calculation_Method="AverageOverDaysInFrequency", Calculation_Frequency=20, Maximum_Figures_in_Shadow_Overlap_Calculations=1000)

# The following classes within the IDF file are kept as they were when generated from Rhino/Grasshopper/Honeybee. These classes describe the geometry for each zone and as such are a pain in the ass to modify manually without risking geometry that isn't possible to simulate.
# "GLOBALGEOMETRYRULES"
# "SHADING:BUILDING:DETAILED"
# "BUILDINGSURFACE:DETAILED"
# "FENESTRATIONSURFACE:DETAILED"
# "ZONE"

## MATERIALS AND CONSTRUCTIONSD TO BE ADJUSTED
# "WINDOWMATERIAL:GAS"
# "WINDOWMATERIAL:GLAZING"
# "MATERIAL:AIRGAP"
# "MATERIAL"
# "CONSTRUCTION"

## SCHEDULES TO BE ADJUSTED
# "SCHEDULETYPELIMITS"
# "SCHEDULE:WEEK:DAILY"
# "SCHEDULE:YEAR"
# "SCHEDULE:DAY:INTERVAL"

## ZONE CONDITIONS TO BE ADJUSTED
# "HVACTEMPLATE:THERMOSTAT"
# "PEOPLE"
# "ELECTRICEQUIPMENT"

## TO KEEP "SIZING:PARAMETERS"
# ENABLE MODIFICATION "LIGHTS"

## UNKNOWN!
# "DESIGNSPECIFICATION:OUTDOORAIR"
# "ZONEINFILTRATION:DESIGNFLOWRATE"
# "ZONEMIXING"
# "HVACTEMPLATE:ZONE:IDEALLOADSAIRSYSTEM"


# ENABLE MODIFICATION 
    


SHADOWCALCULATION,
    AverageOverDaysInFrequency,    !- Calculation Method
    30,                       !- Calculation Frequency
    300,                      !- Maximum Figures in Shadow Overlap Calculations
    SutherlandHodgman,        !- Polygon Clipping Algorithm
    SimpleSkyDiffuseModeling,    !- Sky Diffuse Modeling Algorithm
    InternalCalculation,      !- External Shading Calculation Method
    No;                       !- Output External Shading Calculation Results


In [16]:
# Load a JSON containing a set of room setpoints, gains and schedules for application to IDF zones
internal_gains_library = loadJSON("internal_gains_library.json")

internal_gains_library.keys()

[u'General Teaching',
 u'Computer Lab',
 u'Office Open',
 u'Research Lab',
 u'Office Cellular']

# Below this = TESTING/NOTES

In [20]:
# Load IDF into a list of lists (using basic text)

idf_filepath = "test.idf"

def isplit(iterable, splitters):
    return [list(g) for k,g in itertools.groupby(iterable,lambda x:x in splitters) if not k]

def nestrepl(lst, what, repl):
    for index, item in enumerate(lst):
        if type(item) == list:
            nestrepl(item, what, repl)
        else:
            if item == what:
                lst[index] = repl

with open(idf_filepath, "r") as f:
    content = isplit(isplit([re.sub("[ ]{2,}", "", re.sub("\r\n", "", re.sub("\t", "", x))) for x in f.readlines()], "")[0], "\n")
    content = [[x.replace('\n','') for x in l] for l in content]


* If length of sublist is 1 then only one value e.g. Timestep
* If sublist length of >1 then multiple values
* If count of sublist unique first values (if > 1) 


In [ ]:
Modify the following entries:


In [25]:
# Remove unecessary parts
a = []
for i in content:
    if i[0] not in ["SizingPeriod:DesignDay,"]:
        a.append(i)
    else:
        pass
a

[['Version, ;'],
 ['Timestep, 6;'],
 ['ShadowCalculation,',
  'AverageOverDaysInFrequency,!- Calculation Method',
  '30,!- Calculation Frequency',
  '3000;!- Maximum Figures in Shadow Overlap Calculation'],
 ['Building,',
  'test.idf, !- Name',
  '0.0, !- North Axis {deg}',
  'City, !- Terrain',
  ', !- Loads Convergence Tolerance Value',
  ', !- Temperature Convergence Tolerance Value {deltaC}',
  'FullInteriorAndExteriorWithReflections, !- Solar Distribution or maybe FullExterior',
  ', !- Maximum Number of Warmup Days',
  '; !- Minimum Number of Warmup Days'],
 ['Sizing:Parameters,',
  '1.25,!- Heating Sizing Factor',
  '1.15;!- Cooling Sizing Factor'],
 ['Site:Location,',
  'LONDON/GATWICKGBR,',
  '51.15,!Latitude',
  '-0.18,!Longitude',
  '0.0, !Time Zone',
  '62.0;!Elevation'],
 ['SimulationControl,',
  'Yes,!- Do Zone Sizing Calculation',
  'Yes,!- Do System Sizing Calculation',
  'Yes,!- Do Plant Sizing Calculation',
  'No,!- Run Simulation for Sizing Periods',
  'Yes; !- Run S

In [22]:
[i[0] for i in content]

['Version, ;',
 'Timestep, 6;',
 'ShadowCalculation,',
 'Building,',
 'Sizing:Parameters,',
 'Site:Location,',
 'SizingPeriod:DesignDay,',
 'SizingPeriod:DesignDay,',
 'SizingPeriod:DesignDay,',
 'SizingPeriod:DesignDay,',
 'SizingPeriod:DesignDay,',
 'SizingPeriod:DesignDay,',
 'SizingPeriod:DesignDay,',
 'SimulationControl,',
 'RunPeriod,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'ScheduleTypeLimits,',
 'Sched